# Lectura PDFs congreso

In [1]:
#Importando librerías
import pdfplumber
import pandas as pd
import regex as re
import logging
import os

In [2]:
logging.getLogger("pdfminer").setLevel(logging.ERROR)

In [3]:
#Procesando carpeta entera
relative_destination = '2024/ORDINARIA/S_40_F2'
#output_folder = '../data/votos_procesado/sesion_54/'
session_type = 'ORDINARIA'

In [4]:
#Asignando carpetas
source_folder = f'./RAW-DATA/{relative_destination}'
output_folder_csv = f'./CURATED-DATA/csv/{relative_destination}'
output_folder_parquet = f'./CURATED-DATA/parquet/{relative_destination}'

In [5]:
#Apagando warnings
import warnings
warnings.filterwarnings("ignore")

In [6]:
# Nueva función para extraer el encabezado
def extract_header_text(page):
    text = page.extract_text()
    text = text.replace('\n', ' ')
    #Replacing parentesis
    text = re.sub(r"[()]", "", text)
    #print(text)
    # Extraer el número de evento de votación
    event_number_match = re.search(r'EVENTO DE VOTACIÓN # (\d+)', text)
    event_number = event_number_match.group(1).strip() if event_number_match else None

    # Extraer estampa de tiempo
    timestamp_match = re.search(r'\d{2}-\d{2}-\d{4} \d{2}:\d{2}:\d{2}', text)
    timestamp = timestamp_match.group().strip() if timestamp_match else None

    # Extraer el número de sesión
    session_number_match = re.search(r'SESIÓN No. (\d+)', text)
    session_number = session_number_match.group(1).strip() if session_number_match else None

    # Preparando para extraer nombre de la votación
    text = re.sub(' \d{2}-\d{2}-\d{4} \d{2}:\d{2}:\d{2}', '', text)
    text = re.sub('EVENTO DE VOTACIÓN # \d+', '', text)
    #print(text)

    # Extraer el nombre de la votación
    voting_name_match = re.search(r'Fecha y Hora: ([A-Za-z0-9ÑÁÉÍÓÚÜñáéíóúü\s-]+) SESIÓN No', text)
    voting_name = voting_name_match.group(1).strip() if voting_name_match else None
    #Print if none
    #if voting_name is None:
        #print(text)


    return event_number, timestamp, session_number, voting_name



In [7]:
def clean_column_text(df):
    # Reemplazar saltos de línea en cada columna por un espacio
    df = df.applymap(lambda x: x.replace('\n', ' ') if isinstance(x, str) else x)
    return df

def standardize_columns(df, required_columns):
    # Filtrar las columnas que están en el DataFrame
    existing_columns = [col for col in required_columns if col in df.columns]
    
    # Agregar las columnas faltantes con valores vacíos (None)
    for col in required_columns:
        if col not in df.columns:
            df[col] = None
    
    # Reordenar el DataFrame para que tenga solo las columnas requeridas en el orden correcto
    df = df[required_columns]
    
    return df

def filter_summary_rows(df):
    # Usar una regex para eliminar filas que son resúmenes (como "A favor 143" o "En contra 10")
    regex = r'^[A-Za-z\s]+ \d+$'
    # Filtrar filas que contengan este patrón en cualquier columna
    df = df[~df.apply(lambda row: row.astype(str).str.contains(regex, na=False).any(), axis=1)]
    return df

def read_pdf_to_df(pdf_path, num_columns):
    # Abrir el PDF con pdfplumber
    with pdfplumber.open(pdf_path) as pdf:
        all_dataframes = []
        # Extraer información de encabezado
        event_number, timestamp, session_number, voting_name = extract_header_text(pdf.pages[0])
        print(event_number, timestamp, session_number, voting_name)
        # Excluir la última página (omitimos la última con [:-1])
        for page in pdf.pages[:-1]:
            # Extraer las tablas en formato de lista de listas
            tables = page.extract_tables()
            for table in tables:
                if table:
                    # Convertir la tabla en un DataFrame
                    df = pd.DataFrame(table[1:], columns=table[0])

                    # Limpiar los saltos de línea y eliminar columnas vacías
                    df = clean_column_text(df)

                    # Estandarizar el número de columnas
                    df = standardize_columns(df, num_columns)

                    # Resetear el índice
                    df = df.reset_index(drop=True)

                    # Eliminar columnas completamente vacías (que causan los NaN)
                    df = df.dropna(axis=1, how='all')

                    # Filtrar filas de resumen como "A favor 143" o "En contra 10"
                    df = filter_summary_rows(df)

                    # Agregando columnas de encabezado
                    df['Evento'] = event_number
                    df['Estampa'] = timestamp
                    df['Sesion'] = session_number
                    df['Votacion'] = voting_name

                    # Agregar el DataFrame a la lista
                    all_dataframes.append(df)
        
        # Concatenar todos los DataFrames asegurando que no haya problemas con los índices
        final_df = pd.concat(all_dataframes, ignore_index=True)
        final_df = final_df[~final_df['NOMBRE'].isnull()]
    
    return final_df

In [8]:
# Crear output_folder si no existe
os.makedirs(output_folder_csv, exist_ok=True)
os.makedirs(output_folder_csv, exist_ok=True)

# Obtener PDFs solo un nivel más abajo
source_files = [
    os.path.join(subfolder, file)
    for subfolder in os.listdir(source_folder)
    if os.path.isdir(os.path.join(source_folder, subfolder))
    for file in os.listdir(os.path.join(source_folder, subfolder))
    if file.endswith('.pdf')
]

# Columnas requeridas
required_columns = ['No.', 'NOMBRE', 'BLOQUE', 'VOTO EMITIDO']

# Iterar sobre los archivos
for file in source_files:
    print(f"Procesando {file}")
    
    filepath = os.path.join(source_folder, file)
    df = read_pdf_to_df(filepath, required_columns)

    # Guardar solo con el nombre del archivo, no con la ruta completa
    output_file_name = os.path.join(output_folder_csv, os.path.basename(file).replace('.pdf', '.csv'))
    df.to_csv(output_file_name, index=False)

Procesando Fase2/votacion_pregunta_ELECCIÓN DE GUSTAVO ADOLFO DUBÓN GÁLVEZ COMO MAGISTRADO TITULAR DE LA CORTE DE APELACIONES.pdf
24 08-10-2024 13:10:11 40 ELECCIÓN DE GUSTAVO ADOLFO DUBÓN GÁLVEZ COMO MAGISTRADO TITULAR DE LA CORTE DE APELACIONES
Procesando Fase2/votacion_pregunta_ELECCIÓN DE VÍCTOR MANUEL CASTILLO MAYÉN COMO MAGISTRADO TITULAR DE LA CORTE DE APELACIONES.pdf
279 08-10-2024 18:10:09 40 ELECCIÓN DE VÍCTOR MANUEL CASTILLO MAYÉN COMO MAGISTRADO TITULAR DE LA CORTE DE APELACIONES
Procesando Fase2/votacion_pregunta_ELECCIÓN DE ELSA NOEMÍ FALLA ALONZO COMO MAGISTRADO TITULAR DE LA CORTE DE APELACIONES.pdf
156 08-10-2024 16:10:19 40 ELECCIÓN DE ELSA NOEMÍ FALLA ALONZO COMO MAGISTRADO TITULAR DE LA CORTE DE APELACIONES
Procesando Fase2/votacion_pregunta_ELECCIÓN DE HUGO RENÉ COYOY SAC COMO MAGISTRADO TITULAR DE LA CORTE DE APELACIONES.pdf
131 08-10-2024 15:10:29 40 ELECCIÓN DE HUGO RENÉ COYOY SAC COMO MAGISTRADO TITULAR DE LA CORTE DE APELACIONES
Procesando Fase2/vota

In [9]:
#Lista de archivos guardados
output_files = [file for file in os.listdir(output_folder_csv) if file.endswith('.csv')]
output_files

['votacion_pregunta_ELECCIÓN DE DEISY MARISOL POP CHIQUÍN COMO MAGISTRADO TITULAR DE LA CORTE DE APELACIONES.csv',
 'votacion_pregunta_ELECCIÓN DE LUIS FERNANDO UCLÉS GONZALEZ COMO MAGISTRADO TITULAR DE LA CORTE DE APELACIONES.csv',
 'votacion_pregunta_ELECCIÓN DE NÉSTOR GABRIEL BATRES OCHOA COMO MAGISTRADO TITULAR DE LA CORTE DE APELACIONES.csv',
 'votacion_pregunta_ELECCIÓN DE ERIC NEPTALI GODINEZ MIRANDA COMO MAGISTRADO SUPLENTE DE LA CORTE DE APELACIONES.csv',
 'votacion_pregunta_ELECCIÓN DE MARCO ESTUARDO ORDÓÑEZ GARCÍA COMO MAGISTRADO TITULAR DE LA CORTE DE APELACIONES.csv',
 'votacion_pregunta_ELECCIÓN DE LUIS ANTONIO MORALES DEL CID COMO MAGISTRADO TITULAR DE LA CORTE DE APELACIONES.csv',
 'votacion_pregunta_ELECCIÓN DE MARIA CECILIA DE LEON TERRON COMO MAGISTRADO TITULAR DE LA CORTE DE APELACIONES.csv',
 'votacion_pregunta_ELECCIÓN DE JESSIE ESTELA GUADALUPE PORTILLO DE LEÓN COMO MAGISTRADO TITULAR DE LA CORTE DE APELACIONES.csv',
 'votacion_pregunta_ELECCIÓN D